In [59]:
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent = 4)

df = pd.read_csv('./sequence_categoryok.csv')

columnlist = df.columns.tolist()


In [60]:
#delete rows with no tasks
xy = df[df.Tasks.notnull()]

# for rows in df:
#     if Category is null:
#         category = category from row above
        
xy['Category'].fillna(method='ffill', inplace=True)

xy['Questions'].fillna(method='ffill', inplace=True)   
xy['Field'].fillna(method='ffill', inplace=True) 

xy

,Category,Questions,Field,Tasks,Assignment,Visibility,Time,Min Time FORMULA,Budget
0,General,Wedding (Event) Date,text,Discuss wedding weekend events with client,Planner,Yes,7 months,5,NaN
1,General,Wedding (Event) Date,text,Register for gifts,Client,Yes,7 months,5,NaN
2,General,Wedding (Event) Date,text,- Obtain your marriage license,Client,Yes,3 months,1,NaN
3,General,Wedding (Event) Date,text,"Purchase gifts for wedding party, parents and ...",Client,Yes,2 months,1,NaN
8,General,Guest Count,text,Compile Guest list,Client,Yes,9 months,6,NaN
9,General,Guest Count,text,- Collect Guest email and physical addresses,Client,Yes,8 months,6,NaN
10,General,Guest Count,text,- RSVP deadline,Planner,Yes,1 Month,1,NaN
11,General,Guest Count,text,- Send Final guest count to Planner,Client,Yes,1 month,2 weeks,NaN
12,General,Guest Count,text,- Update floor plan based on final guest count...,Planner,No,4 weeks,4 weeks,NaN
13,General,Guest Count,text,Final table/seat assignments due,Client,Yes,1 month,3 weeks,NaN


In [63]:
description = xy['Tasks']
question = [x for x in xy['Questions']]
field = [x for x in xy['Field']]
timing = [x for x in xy['Time']]
mintime = [x for x in xy['Min Time FORMULA']]
assignment = [x for x in xy['Assignment']]
client = [x for x in xy['Visibility']]
category = [x for x in xy['Category']]

#TODO get rid of space after description
description = [x.replace("- ", "", 1) for x in description]


#need answer onl yes no or maybe
#timing needs to be 
for i in range(len(timing)):
    timing_num = int(''.join(filter(str.isdigit, timing[i])))
    mintime_num = mintime[i]
    startletter= ''.join(filter(lambda x: x.isalpha(), timing[i]))[0]
    
            
    if startletter == 'm':
        timing[i] = timing_num * 30
        mintime[i] = mintime_num * 30
    elif startletter == 'w':
        timing[i] = timing_num * 7
        mintime[i] = mintime_num * 7
    elif startletter == 'd':
        timing[i] = timing_num
        mintime[i] = mintime_num
    timing[i] *= -1
    mintime[i] *= -1

            
            
print(timing)
print(mintime)
        
for i, c in enumerate(client):
    if c != 'yes':
        client[i] = 'no'

# pairs = [[category[i], questions[i]] for i in range(len(category))]

['Discuss wedding weekend events with client', 'Register for gifts ', 'Obtain your marriage license ', 'Purchase gifts for wedding party, parents and each other', 'Compile Guest list', 'Collect Guest email and physical addresses', 'RSVP deadline', 'Send Final guest count to Planner', 'Update floor plan based on final guest count and send to client for table/seat assignments', 'Final table/seat assignments due', 'Update Contracts with final guest count', 'Update final payments (to reflect final counts)', 'Guest count contractually locked in ', 'Determine budget ', 'Draft a detailed budget to determine spend', 'Review estimated event budget', 'Set up wedding website', 'Add date, wedding event location/time, and hotel block information to Wedding Website', "Send wedding website info (how to's)", 'Add transportation and additional event info to wedding website', 'Share Pinterest Board and inspiration with planner', 'Send recommendations for wedding party attire', 'Book appointements/Shop f

TypeError: 'float' object is not iterable

In [3]:
#added on variables
timingratio = [-x/300  for x in timing]
print(timingratio)

[0.23333333333333334, 0.09333333333333334, 0.1, 0.2, 0.04666666666666667, 1.0, 1.0, 0.03, 0.03333333333333333, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 0.9, 0.9, 0.9, 0.9]


In [4]:

  
# Calling DataFrame constructor after zipping 
# both lists, with columns specified 
df = pd.DataFrame(list(zip(description, question, field, timing, timingratio, mintime, assignment, client, category)), 
               columns =['description', 'question', 'field', 'timing', 'timingratio', 'mintime', 'assignment', 'client', 'category']) 
df 




,description,question,field,timing,timingratio,mintime,assignment,client,category
0,Pull together venue referrals and set up tours,Is the Venue booked?,No,-70,0.233333,-42.0,planner,no,Venue
1,Review venue referrals and provide feedback,Is the Venue booked?,No,-28,0.093333,-42.0,client,yes,Venue
2,Finalize and book venue,Is the Venue booked?,No,-30,0.100000,-180.0,planner,no,Venue
3,Check if venue has preferred vendor requirements,Is the Venue booked?,No,-60,0.200000,-150.0,planner,no,Venue
4,Enter venue(s) information,Is the Venue booked?,No,-14,0.046667,-42.0,planner,no,Venue
5,Confirm venue delivery and pick up requirements,Is the Venue booked?,No,-300,1.000000,-150.0,planner,no,Venue
6,Request venue contract from client,Is the Venue booked?,Yes,-300,1.000000,-180.0,planner,no,Venue
7,Send venue contracts and pertinent information...,Is the Venue booked?,Yes,-9,0.030000,-6.0,planner,no,Venue
8,Enter venue(s) information,Is the Venue booked?,Yes,-10,0.033333,-6.0,planner,no,Venue
9,Check if venue has preferred vendor requirements,Is the Venue booked?,Yes,-300,1.000000,-150.0,planner,no,Venue


In [5]:
content = df.to_csv()
with open('sample.csv','w') as file:
    print("writing")
    file.write(content)
    file.close()


writing
